In [49]:
import numpy as np

from abc import ABC, abstractmethod

class Process:
    def __init__(self, id, arrival_time, burst_time, priority = "low"):
        self.id = id
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.priority = priority

class Algorithm(ABC):
    @abstractmethod
    def schedule(self, processes):
        pass

class FirstComeFirstServe(Algorithm):
    def schedule(self, processes):
        current_time = 0
        context_switches = 0
        wait_times = []

        for process in processes:
            if current_time < process.arrival_time:
                current_time = process.arrival_time

            wait_time = current_time - process.arrival_time
            wait_times.append(wait_time)

            current_time += process.burst_time
            if process != processes[0]:
                context_switches += 1

        return context_switches, current_time, wait_times
    
class RoundRobin(Algorithm):
    def __init__(self, quantum):
        self.quantum = quantum

    def schedule(self, processes):
        current_time = 0
        context_switches = 0
        process_queue = processes.copy()
        wait_times = [0] * len(processes)
        last_start_time = [0] * len(processes)

        while process_queue:
            process = process_queue.pop(0)

            if current_time < process.arrival_time:
                current_time = process.arrival_time

            # Aktualisiere die Wartezeit jedes Mal, wenn der Prozess ausgeführt wird
            if current_time >= last_start_time[process.id - 1]:
                wait_times[process.id - 1] += current_time - last_start_time[process.id - 1]

            execution_time = min(process.burst_time, self.quantum)
            process.burst_time -= execution_time
            current_time += execution_time
            last_start_time[process.id - 1] = current_time

            if process.burst_time > 0:
                process_queue.append(process)
                context_switches += 1

        return context_switches, current_time, wait_times
    
class MultiLevelQueue(Algorithm):
    def __init__(self, quantum):
        self.quantum = quantum

    def schedule(self, processes):
        # Separate processes by priority
        high_priority = [p for p in processes if p.priority == 'high']
        low_priority = [p for p in processes if p.priority == 'low']

        current_time = 0
        context_switches = 0
        wait_times = [0] * len(processes)
        last_start_time = [0] * len(processes)

        while high_priority or low_priority:
            # Select the next process based on priority and arrival time
            next_process = None
            if high_priority and (not low_priority or high_priority[0].arrival_time <= current_time):
                next_process = high_priority.pop(0)
            elif low_priority and (not high_priority or low_priority[0].arrival_time <= current_time):
                next_process = low_priority.pop(0)

            if next_process:
                if next_process.priority == 'high':
                    wait_times[next_process.id - 1] = current_time - next_process.arrival_time
                    current_time += next_process.burst_time
                else:
                    # Mimic Round Robin scheduling for low priority
                    execution_time = min(next_process.burst_time, self.quantum)
                    if current_time >= last_start_time[next_process.id - 1]:
                        wait_times[next_process.id - 1] += current_time - last_start_time[next_process.id - 1]
                    next_process.burst_time -= execution_time
                    current_time += execution_time
                    last_start_time[next_process.id - 1] = current_time

                    if next_process.burst_time > 0:
                        low_priority.append(next_process)
                        context_switches += 1  # Count context switch for RR

            else:
                current_time += 1

        # Adjust context switches for the first process
        if context_switches > 0 and low_priority:
            context_switches -= 1

        return context_switches, current_time, wait_times

class Scheduler:
    def __init__(self):
        self.processes = []
        self.metrics = {}

    def add_process(self, process):
        self.processes.append(process)

    def run_algorithm(self, algorithm):
        context_switches, current_time, wait_times = algorithm.schedule(self.processes)
        self.calculate_metrics(context_switches, current_time, wait_times)
        self.display_metrics()

    def calculate_metrics(self, context_switches, current_time, wait_times):
        total_turnaround_time = 0
        for process in self.processes:
            total_turnaround_time += wait_times[process.id - 1] + process.burst_time

        average_wait_time = sum(wait_times) / len(self.processes)
        average_turnaround_time = total_turnaround_time / len(self.processes)
        throughput = len(self.processes) / current_time
        fairness_index = np.std(wait_times)

        self.metrics = {
            "average_wait_time": average_wait_time,
            "average_turnaround_time": average_turnaround_time,
            "throughput": throughput,
            "fairness_index": fairness_index,
            "context_switches": context_switches
        }

    def display_metrics(self):
        for metric, value in self.metrics.items():
            print(f"{metric.replace('_', ' ').title()}: {value:.2f} Einheiten")

# Beispiel für das Erstellen von Testdaten
def create_test_processes():
    processes = [
        Process(1, 0, 40, "low"),
        Process(2, 1, 3, "high"),
        Process(3, 2, 2, "low"),
        Process(4, 2, 4, "low"),
        Process(5, 3, 3, "low"),
        Process(6, 3, 2, "high"),
        Process(7, 6, 5, "high"),
        Process(8, 12, 3, "low"),
        Process(9, 12, 2, "high")
        
    ]
    return processes

# Hauptprogramm
scheduler = Scheduler()
test_processes = create_test_processes()
for p in test_processes:
    scheduler.add_process(p)

# Algorithmen ausführen
fcfs = FirstComeFirstServe()
scheduler = Scheduler()
test_processes = create_test_processes()
for p in test_processes:
    scheduler.add_process(p)
scheduler.run_algorithm(fcfs)
print()

rr = RoundRobin(quantum=2)
scheduler = Scheduler()
test_processes = create_test_processes()
for p in test_processes:
    scheduler.add_process(p)
scheduler.run_algorithm(rr)
print()

mlq = MultiLevelQueue(quantum=2)
scheduler = Scheduler()
test_processes = create_test_processes()
for p in test_processes:
    scheduler.add_process(p)
scheduler.run_algorithm(mlq)

Average Wait Time: 40.33 Einheiten
Average Turnaround Time: 47.44 Einheiten
Throughput: 0.14 Einheiten
Fairness Index: 14.68 Einheiten
Context Switches: 8.00 Einheiten

Average Wait Time: 17.89 Einheiten
Average Turnaround Time: 17.89 Einheiten
Throughput: 0.14 Einheiten
Fairness Index: 6.62 Einheiten
Context Switches: 25.00 Einheiten

Average Wait Time: 12.56 Einheiten
Average Turnaround Time: 13.89 Einheiten
Throughput: 0.14 Einheiten
Fairness Index: 10.77 Einheiten
Context Switches: 22.00 Einheiten


In [3]:
# Create samples for boxplot

import random
import numpy as np

class Process:
    def __init__(self, id, arrival_time, burst_time, priority="low"):
        self.id = id
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.priority = priority

    def __repr__(self):
        return f"Process(id={self.id}, arrival_time={self.arrival_time}, burst_time={self.burst_time}, priority='{self.priority}')"

# Constants
    

num_processes = 100
mean_burst_time = 3
std_dev_burst = np.sqrt(5)
percentage_high_priority = 0.1

processes = []

base_arrival_time = 0
for i in range(num_processes):
    burst_time = max(1, int(round(np.random.normal(mean_burst_time, std_dev_burst))))
    priority = "high" if random.random() < percentage_high_priority else "low"
    arrival_time = max(0, int(base_arrival_time))

    process = Process(id=i+1, arrival_time=arrival_time, burst_time=burst_time, priority=priority)
    processes.append(process)

    base_arrival_time += max(0, round(random.uniform(-2, 5)))

processes[:100]  # Displaying first 10 processes for brevity

[Process(id=1, arrival_time=0, burst_time=3, priority='high'),
 Process(id=2, arrival_time=0, burst_time=4, priority='low'),
 Process(id=3, arrival_time=2, burst_time=4, priority='low'),
 Process(id=4, arrival_time=5, burst_time=4, priority='low'),
 Process(id=5, arrival_time=5, burst_time=1, priority='low'),
 Process(id=6, arrival_time=5, burst_time=4, priority='low'),
 Process(id=7, arrival_time=7, burst_time=1, priority='low'),
 Process(id=8, arrival_time=8, burst_time=3, priority='low'),
 Process(id=9, arrival_time=8, burst_time=4, priority='low'),
 Process(id=10, arrival_time=13, burst_time=2, priority='low'),
 Process(id=11, arrival_time=17, burst_time=1, priority='low'),
 Process(id=12, arrival_time=21, burst_time=4, priority='high'),
 Process(id=13, arrival_time=21, burst_time=6, priority='low'),
 Process(id=14, arrival_time=24, burst_time=6, priority='low'),
 Process(id=15, arrival_time=24, burst_time=7, priority='low'),
 Process(id=16, arrival_time=25, burst_time=2, priority=